# Calculate wet bulb temperature

## TODO need to download extra vars for climatology.

- Repeat of DeltaT work but using Wet Bulb
- Would like to pull out population projection code into common module (maybe a class to hold init setup)
- Calculate WBT, WBGT for current and climatology


In [1]:
%matplotlib inline

from pathlib import Path
from datetime import date

import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

import cartopy
from cartopy import crs

import geopandas as gpd 
from config import DATA_SRC, WEATHER_FILE, CLIMATOLOGY_FILE_RESAMP, SHAPEFILES_SRC
import weather_ecmwf
import util
import config



In [8]:
import climate_conversions

In [22]:
wbt = climate_conversions.calculate_wbt(era_weather.temperature_2m, 
                                  era_weather.relative_humidity, 
                                  era_weather.surface_pressure)

/Users/jonathanchambers/Documents/UCL/Lancet/health-climate-exposure/climate_conversions.py:63: RuntimeWarning: invalid value encountered in log
  
